In [3]:
import os
import pandas as pd
import numpy as np
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import re
from tqdm import tqdm
import multiprocessing as mp
import time
import random
import datetime

In [4]:
def loop_slices(path, num_slices=20):
  cnt = 0
  cnt1 = 0
  mpd_playlists = []
  unique_tracks= pd.DataFrame()
  filenames = os.listdir(path)
  for fname in tqdm(sorted(filenames, key=len)):
    if fname.startswith("mpd.slice.") and fname.endswith(".json"):
      cnt += 1
      fullpath = os.sep.join((path, fname))
      f = open(fullpath)
      js = f.read()
      f.close()
      current_slice = json.loads(js)
      # Create a list of all playlists
      for playlist in current_slice['playlists']:
        cnt1 +=1
        mpd_playlists.append(playlist)
        if cnt1 == 1000:
          cnt1=0
          temp=pd.DataFrame(mpd_playlists)
          temp=temp.explode('tracks')
          temp=pd.DataFrame(temp['tracks'].apply(pd.Series))
          unique_tracks=pd.concat([unique_tracks,temp],axis=0)
          unique_tracks.drop_duplicates(subset=['track_uri'],inplace=True)
          mpd_playlists = []
      if cnt == num_slices:
        break
  return unique_tracks
# Path where the json files are extracted


In [9]:
path = 'dset/spotify_million_playlist_dataset/data/'
df = loop_slices(path, num_slices=1000)
df.to_csv('dset/postprocess.csv')

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|█████████▉| 999/1000 [1:11:59<00:04,  4.32s/it]


In [10]:
df["track_uri"] = df["track_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])
df["artist_uri"] = df["artist_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])
df["album_uri"] = df["album_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])

In [ ]:
df = df[['track_uri', 'artist_uri', 'album_uri']]